In [84]:
from __future__ import division
import os, sys, re, json, time, datetime, shutil, csv, glob
import itertools, collections
from importlib import reload
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk
from nltk.tokenize import word_tokenize

# NumPy and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf
assert(tf.__version__.startswith("1.8"))

from common import utils, vocabulary


In [85]:
# Load labeled data set
review_sentences = pd.read_csv('./data/Feature_sentences_Prod_Final_2018-07-09_Labeled_1K.csv', encoding='utf-8')
# Distribution by Sentiment Class
print(review_sentences.groupby('POS_SENTIMENT').size())

POS_SENTIMENT
0    213
1    826
dtype: int64


In [86]:
# Load inital extract for join/matching to obtain ratings
path =r'..' # use your path
allFiles = glob.glob(path + "/data/RR_Pared_All_Active_2018-07-09_*.csv")
all_reviews = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, index_col=None, header=0)
    list_.append(df)
all_reviews = pd.concat(list_)

print(all_reviews.shape)


(369888, 8)


In [87]:
# Use ratings as weak label for training for all sentences:

all_sentences = pd.read_csv('../product_feature_sentences/Feature_sentences_Prod_Final_2018-07-09.csv', encoding='utf-8')
print(all_sentences.shape)


# calculate rating label given many nans
#   (I'm sure there is a better way to do this but . . .)
def calc_rating_label(oall, org, app, p_v, dur):
    avg_numerator = 0.0
    avg_divisor = 0.0
    avg = 0.0
    if np.isfinite(oall):
        avg_numerator += oall
        avg_divisor += 1
    if np.isfinite(org):
        avg_numerator += org
        avg_divisor += 1
    if np.isfinite(app):
        avg_numerator += app
        avg_divisor += 1
    if np.isfinite(p_v):
        avg_numerator += p_v
        avg_divisor += 1
    if np.isfinite(dur):
        avg_numerator += dur
        avg_divisor += 1

    if ((avg_numerator > 0) & (avg_divisor > 0)):
        avg = avg_numerator / avg_divisor
        if avg > 7.9:
            return 1
        else: 
            return 0
    else:
        return 0

review_sentences['WEAK_LABEL'] = 0

for index, row in all_sentences.iterrows():
    overall = all_reviews['OVERALL'][all_reviews['TESTIMONIALID'] == row['TESTIMONIAL_ID']].values[0]
    organization = all_reviews['ORGANIZATION'][all_reviews['TESTIMONIALID'] == row['TESTIMONIAL_ID']].values[0]
    appearance = all_reviews['APPEARANCE'][all_reviews['TESTIMONIALID'] == row['TESTIMONIAL_ID']].values[0]
    price_value = all_reviews['PRICE_VALUE'][all_reviews['TESTIMONIALID'] == row['TESTIMONIAL_ID']].values[0]
    durability = all_reviews['DURABILITY'][all_reviews['TESTIMONIALID'] == row['TESTIMONIAL_ID']].values[0]
    all_sentences.at[index, 'WEAK_LABEL'] = calc_rating_label(overall, organization, appearance, price_value, durability)
    #print(calc_rating_label(overall, organization, appearance, price_value, durability))
    
print(review_sentences.groupby('WEAK_LABEL').size())

(146018, 3)
WEAK_LABEL
0    1039
dtype: int64


In [90]:
dataset_class_0 = review_sentences[review_sentences['WEAK_LABEL'] == 0]
dataset_class_1 = review_sentences[review_sentences['WEAK_LABEL'] == 1]
all_class_0 = all_sentences[all_sentences['WEAK_LABEL'] == 0]
all_class_1 = all_sentences[all_sentences['WEAK_LABEL'] == 1]

# Use entire sentence set for training of weak labels
train = all_class_0.iloc[0:18085,]
train = train.append(all_class_1.iloc[0:127933,])
print('train shape:', train.shape)
print(train.head())

# Use full labeled set for testing of actual labels
test = dataset_class_0.iloc[0:124,]
test = test.append(dataset_class_1.iloc[0:915,])
print('test shape:', test.shape)

train shape: (146018, 4)
    TESTIMONIAL_ID  SENTENCE_ID  \
18         1699837            0   
26          696885            2   
29          795079            1   
30          795079            3   
39         1057624            1   

                                          SENTENCE  WEAK_LABEL  
18                          Easy access protective         0.0  
26      The color is true light weight and durable         0.0  
29                      Very nice sturdy and roomy         0.0  
30                    This bag is the perfect size         0.0  
39  Good quality and fits comfortably around waist         0.0  
test shape: (124, 5)


In [91]:
train_x = train['SENTENCE']
train_y = train['WEAK_LABEL']
test_x = test['SENTENCE']
test_y = test['POS_SENTIMENT']

In [92]:
# Tokenize sentences

train_x_tokens = []
test_x_tokens = []

for x in train_x:
    train_x_tokens.append(word_tokenize(x))
    
for x in test_x:
    test_x_tokens.append(word_tokenize(x))

In [93]:
# define vocabulary using w266 common vocab

train_text_all = [item for sublist in train_x_tokens for item in sublist]

vocab = vocabulary.Vocabulary(train_text_all, size=20000)
print("Vocabulary size: {:,}".format(vocab.size))
#print("Vocabulary dict: ", vocab.word_to_id)

train_x_ids = []
test_x_ids = []

for x in train_x_tokens:
    train_x_ids.append(vocab.words_to_ids(x))
    
for x in test_x_tokens:
    test_x_ids.append(vocab.words_to_ids(x))

Vocabulary size: 7,922


In [ ]:
# NB - BoW Baseline

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

train_x_fdict = []
test_x_fdict = []

for x in train_x_ids:
    train_x_fdict.append(collections.Counter(x))
    
for x in test_x_ids:
    test_x_fdict.append(collections.Counter(x))

train_x_vector = []
test_x_vector = []

num_features = vocab.size
print('creating id lists...')
for x in train_x_fdict:
    train_x_vector.append([x.get(i, 0) for i in range(num_features)])
    
for x in test_x_fdict:
    test_x_vector.append([x.get(i, 0) for i in range(num_features)])

# use w266 common utils to convert id lists to sparse bow matrix
print('starting id lists to sparse bow conversion...')
train_x_sparse_bow = utils.id_lists_to_sparse_bow(train_x_fdict, vocab.size)
test_x_sparse_bow = utils.id_lists_to_sparse_bow(test_x_fdict, vocab.size)

# training Multinomial Naive Bayes for simple baseline model
print('training Multinomial Naive Bayes for simple baseline model...')
nb = MultinomialNB()
nb.fit(train_x_sparse_bow, train_y)
y_pred = nb.predict(test_x_sparse_bow)

acc = accuracy_score(test_y, y_pred)
print("Accuracy on test set: {:.02%}".format(acc))
classrep = classification_report(test_y, y_pred)
print(classrep)

creating id lists...


In [73]:
# Preprocessing for CNN model

from sklearn import preprocessing
# pad all id vectors to max length for Neural model

max_comment_length = max([len(x) for x in train_x_ids])

train_x_neural = train_x_ids
test_x_neural = test_x_ids

for x in train_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]
        
for x in test_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]

# convert labels to one-hot vectors for Neural model --update your class lables below!!

terms_list = [[0],[1]]
lb = preprocessing.MultiLabelBinarizer()
lb.fit(terms_list)

train_labels = np.array(train_y.tolist())
test_labels = np.array(test_y.tolist())

train_lables_format = [train_labels[i:i+1] for i in range(0, len(train_labels), 1)]    
test_lables_format = [test_labels[i:i+1] for i in range(0, len(test_labels), 1)]

train_y_neural = lb.transform(train_lables_format)
test_y_neural = lb.transform(test_lables_format)

In [74]:
from Text_CNN import TextCNN

# CNN Model
# Leveraged Denny Britz template for CNN:
# http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/

# run this to clear tf flags without re-starting kernel

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)
        
del_all_flags(tf.flags.FLAGS)

# Parameters
# ==================================================

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.app.flags.DEFINE_string('f', '', 'kernel')

FLAGS = tf.flags.FLAGS
FLAGS(sys.argv)
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Data Preparation
# ==================================================

x_train = np.array(train_x_neural)
y_train = np.array(train_y_neural)
x_dev = np.array(test_x_neural)
y_dev = np.array(test_y_neural)

print("x_train", np.shape(x_train))
print("y_train", np.shape(y_train))
print("x_dev", np.shape(x_dev))
print("x_dev", np.shape(y_dev))

print("Vocabulary Size: {:d}".format(vocab.size))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))


# Training
# ==================================================

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]
            
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=2,
            vocab_size=vocab.size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))




Parameters:
ALLOW_SOFT_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f14fda9b710>
BATCH_SIZE=<absl.flags._flag.Flag object at 0x7f14fda9bf60>
CHECKPOINT_EVERY=<absl.flags._flag.Flag object at 0x7f14fda9bc88>
DROPOUT_KEEP_PROB=<absl.flags._flag.Flag object at 0x7f14fda9bf28>
EMBEDDING_DIM=<absl.flags._flag.Flag object at 0x7f15021ac0b8>
EVALUATE_EVERY=<absl.flags._flag.Flag object at 0x7f14fda9bd30>
F=<absl.flags._flag.Flag object at 0x7f14fdc14f98>
FILTER_SIZES=<absl.flags._flag.Flag object at 0x7f15021ac048>
L2_REG_LAMBDA=<absl.flags._flag.Flag object at 0x7f14fda9be48>
LOG_DEVICE_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f14fda9b940>
NUM_CHECKPOINTS=<absl.flags._flag.Flag object at 0x7f14fda9bbe0>
NUM_EPOCHS=<absl.flags._flag.Flag object at 0x7f14fda9beb8>
NUM_FILTERS=<absl.flags._flag.Flag object at 0x7f14fda9bef0>

x_train (1039, 71)
y_train (1039, 2)
x_dev (1039, 71)
x_dev (1039, 2)
Vocabulary Size: 1890
Train/Dev split: 1039/1039
INFO:tensorflow:Summary name em

2018-08-10T21:52:00.749837: step 83, loss 0.55832, acc 0.8125
2018-08-10T21:52:00.898819: step 84, loss 0.349921, acc 0.875
2018-08-10T21:52:00.947301: step 85, loss 0.295278, acc 0.866667
2018-08-10T21:52:01.096608: step 86, loss 0.526333, acc 0.828125
2018-08-10T21:52:01.244094: step 87, loss 0.388704, acc 0.859375
2018-08-10T21:52:01.391934: step 88, loss 0.403693, acc 0.84375
2018-08-10T21:52:01.542984: step 89, loss 0.525245, acc 0.828125
2018-08-10T21:52:01.698637: step 90, loss 0.279836, acc 0.828125
2018-08-10T21:52:01.847282: step 91, loss 0.590743, acc 0.796875
2018-08-10T21:52:01.993898: step 92, loss 0.256284, acc 0.921875
2018-08-10T21:52:02.142873: step 93, loss 0.364287, acc 0.875
2018-08-10T21:52:02.289786: step 94, loss 0.403997, acc 0.875
2018-08-10T21:52:02.435976: step 95, loss 0.30747, acc 0.890625
2018-08-10T21:52:02.583257: step 96, loss 0.323862, acc 0.90625
2018-08-10T21:52:02.728139: step 97, loss 0.212702, acc 0.90625
2018-08-10T21:52:02.877280: step 98, loss

2018-08-10T21:52:19.367438: step 205, loss 0.149516, acc 0.9375
2018-08-10T21:52:19.516232: step 206, loss 0.0454261, acc 1
2018-08-10T21:52:19.664546: step 207, loss 0.182141, acc 0.921875
2018-08-10T21:52:19.812500: step 208, loss 0.0743185, acc 0.96875
2018-08-10T21:52:19.960444: step 209, loss 0.215738, acc 0.953125
2018-08-10T21:52:20.107329: step 210, loss 0.212364, acc 0.96875
2018-08-10T21:52:20.256348: step 211, loss 0.0480911, acc 0.96875
2018-08-10T21:52:20.403909: step 212, loss 0.182499, acc 0.9375
2018-08-10T21:52:20.552619: step 213, loss 0.192562, acc 0.953125
2018-08-10T21:52:20.700121: step 214, loss 0.197807, acc 0.953125
2018-08-10T21:52:20.847405: step 215, loss 0.179259, acc 0.953125
2018-08-10T21:52:20.993852: step 216, loss 0.177155, acc 0.921875
2018-08-10T21:52:21.139382: step 217, loss 0.165839, acc 0.9375
2018-08-10T21:52:21.284914: step 218, loss 0.183197, acc 0.953125
2018-08-10T21:52:21.434085: step 219, loss 0.0924141, acc 0.96875
2018-08-10T21:52:21.579

2018-08-10T21:52:37.542231: step 328, loss 0.0635645, acc 0.96875
2018-08-10T21:52:37.684598: step 329, loss 0.0581777, acc 0.984375
2018-08-10T21:52:37.828613: step 330, loss 0.016759, acc 1
2018-08-10T21:52:37.975585: step 331, loss 0.0814226, acc 0.96875
2018-08-10T21:52:38.123046: step 332, loss 0.0724513, acc 0.96875
2018-08-10T21:52:38.267983: step 333, loss 0.0734347, acc 0.96875
2018-08-10T21:52:38.416826: step 334, loss 0.0738033, acc 0.96875
2018-08-10T21:52:38.564924: step 335, loss 0.0521471, acc 0.984375
2018-08-10T21:52:38.710112: step 336, loss 0.0383339, acc 0.984375
2018-08-10T21:52:38.856298: step 337, loss 0.105025, acc 0.96875
2018-08-10T21:52:39.004058: step 338, loss 0.0296881, acc 0.984375
2018-08-10T21:52:39.150054: step 339, loss 0.0272038, acc 0.984375
2018-08-10T21:52:39.196723: step 340, loss 0.031355, acc 1
2018-08-10T21:52:39.348633: step 341, loss 0.0833616, acc 0.953125
2018-08-10T21:52:39.498845: step 342, loss 0.0179591, acc 1
2018-08-10T21:52:39.64893

2018-08-10T21:52:56.011784: step 453, loss 0.00327848, acc 1
2018-08-10T21:52:56.152185: step 454, loss 0.0217916, acc 0.984375
2018-08-10T21:52:56.292121: step 455, loss 0.079457, acc 0.96875
2018-08-10T21:52:56.434433: step 456, loss 0.027527, acc 0.984375
2018-08-10T21:52:56.575249: step 457, loss 0.0333143, acc 0.984375
2018-08-10T21:52:56.714260: step 458, loss 0.0101768, acc 1
2018-08-10T21:52:56.757069: step 459, loss 0.00372353, acc 1
2018-08-10T21:52:56.898540: step 460, loss 0.0085958, acc 1
2018-08-10T21:52:57.037332: step 461, loss 0.0303064, acc 0.984375
2018-08-10T21:52:57.179262: step 462, loss 0.0651837, acc 0.96875
2018-08-10T21:52:57.317218: step 463, loss 0.0127081, acc 1
2018-08-10T21:52:57.459535: step 464, loss 0.00730329, acc 1
2018-08-10T21:52:57.595746: step 465, loss 0.0108088, acc 1
2018-08-10T21:52:57.735641: step 466, loss 0.0239526, acc 1
2018-08-10T21:52:57.875662: step 467, loss 0.0133126, acc 1
2018-08-10T21:52:58.017188: step 468, loss 0.0450289, acc 1

...

2018-08-10T21:58:55.830685: step 2878, loss 0.000252491, acc 1
2018-08-10T21:58:55.975626: step 2879, loss 9.22516e-05, acc 1
2018-08-10T21:58:56.122167: step 2880, loss 0.000358546, acc 1
2018-08-10T21:58:56.268025: step 2881, loss 0.000602389, acc 1
2018-08-10T21:58:56.414777: step 2882, loss 6.7997e-05, acc 1
2018-08-10T21:58:56.558249: step 2883, loss 0.000265905, acc 1
2018-08-10T21:58:56.703428: step 2884, loss 0.000514598, acc 1
2018-08-10T21:58:56.851426: step 2885, loss 0.000237512, acc 1
2018-08-10T21:58:56.997393: step 2886, loss 0.000539879, acc 1
2018-08-10T21:58:57.145804: step 2887, loss 0.000449998, acc 1
2018-08-10T21:58:57.294875: step 2888, loss 0.000356688, acc 1
2018-08-10T21:58:57.446254: step 2889, loss 0.000468163, acc 1
2018-08-10T21:58:57.493781: step 2890, loss 6.08745e-05, acc 1
2018-08-10T21:58:57.644636: step 2891, loss 0.000268185, acc 1
2018-08-10T21:58:57.791056: step 2892, loss 0.000393344, acc 1
2018-08-10T21:58:57.938665: step 2893, loss 8.20183e-05,

2018-08-10T21:59:14.775537: step 3002, loss 0.00103484, acc 1
2018-08-10T21:59:14.917809: step 3003, loss 0.0016391, acc 1
2018-08-10T21:59:15.070085: step 3004, loss 7.19519e-05, acc 1
2018-08-10T21:59:15.214938: step 3005, loss 0.000143971, acc 1
2018-08-10T21:59:15.361046: step 3006, loss 0.00116521, acc 1
2018-08-10T21:59:15.513900: step 3007, loss 9.22515e-05, acc 1
2018-08-10T21:59:15.663763: step 3008, loss 0.000182338, acc 1
2018-08-10T21:59:15.709231: step 3009, loss 0.000545795, acc 1
2018-08-10T21:59:15.855330: step 3010, loss 0.000622237, acc 1
2018-08-10T21:59:16.005188: step 3011, loss 0.000196729, acc 1
2018-08-10T21:59:16.152081: step 3012, loss 0.00480928, acc 1
2018-08-10T21:59:16.298848: step 3013, loss 0.000397866, acc 1
2018-08-10T21:59:16.445380: step 3014, loss 0.000422942, acc 1
2018-08-10T21:59:16.590702: step 3015, loss 0.000755768, acc 1
2018-08-10T21:59:16.736455: step 3016, loss 9.32166e-05, acc 1
2018-08-10T21:59:16.885952: step 3017, loss 0.00237441, acc 

2018-08-10T21:59:33.552587: step 3129, loss 0.0002571, acc 1
2018-08-10T21:59:33.698459: step 3130, loss 0.00199535, acc 1
2018-08-10T21:59:33.838751: step 3131, loss 0.000256764, acc 1
2018-08-10T21:59:33.984741: step 3132, loss 9.14902e-05, acc 1
2018-08-10T21:59:34.126524: step 3133, loss 0.000313734, acc 1
2018-08-10T21:59:34.264927: step 3134, loss 0.000200692, acc 1
2018-08-10T21:59:34.403871: step 3135, loss 0.00454878, acc 1
2018-08-10T21:59:34.541947: step 3136, loss 0.00257716, acc 1
2018-08-10T21:59:34.681451: step 3137, loss 8.36747e-05, acc 1
2018-08-10T21:59:34.822387: step 3138, loss 0.00136213, acc 1
2018-08-10T21:59:34.960946: step 3139, loss 7.67904e-05, acc 1
2018-08-10T21:59:35.099639: step 3140, loss 0.0134095, acc 1
2018-08-10T21:59:35.239946: step 3141, loss 0.000475253, acc 1
2018-08-10T21:59:35.381057: step 3142, loss 0.000156991, acc 1
2018-08-10T21:59:35.526521: step 3143, loss 0.000105672, acc 1
2018-08-10T21:59:35.669885: step 3144, loss 7.05172e-05, acc 1


2018-08-10T21:59:52.551745: step 3258, loss 0.000183116, acc 1
2018-08-10T21:59:52.694034: step 3259, loss 0.000623993, acc 1
2018-08-10T21:59:52.832550: step 3260, loss 0.000350203, acc 1
2018-08-10T21:59:52.973168: step 3261, loss 0.00361969, acc 1
2018-08-10T21:59:53.114215: step 3262, loss 0.000703531, acc 1
2018-08-10T21:59:53.251029: step 3263, loss 0.000190858, acc 1
2018-08-10T21:59:53.293839: step 3264, loss 7.92546e-05, acc 1
2018-08-10T21:59:53.441146: step 3265, loss 0.000468494, acc 1
2018-08-10T21:59:53.578175: step 3266, loss 0.000117135, acc 1
2018-08-10T21:59:53.721173: step 3267, loss 9.98416e-05, acc 1
2018-08-10T21:59:53.859490: step 3268, loss 0.000262768, acc 1
2018-08-10T21:59:54.000256: step 3269, loss 0.000232309, acc 1
2018-08-10T21:59:54.139063: step 3270, loss 0.00125115, acc 1
2018-08-10T21:59:54.278867: step 3271, loss 0.00166102, acc 1
2018-08-10T21:59:54.424187: step 3272, loss 0.00433812, acc 1
2018-08-10T21:59:54.563113: step 3273, loss 0.000841069, ac

2018-08-10T22:00:11.197151: step 3386, loss 7.71693e-05, acc 1
2018-08-10T22:00:11.347147: step 3387, loss 0.000213408, acc 1
2018-08-10T22:00:11.498617: step 3388, loss 6.95814e-05, acc 1
2018-08-10T22:00:11.649900: step 3389, loss 0.000272925, acc 1
2018-08-10T22:00:11.798350: step 3390, loss 0.00140401, acc 1
2018-08-10T22:00:11.952251: step 3391, loss 0.000258089, acc 1
2018-08-10T22:00:12.101104: step 3392, loss 0.0169466, acc 0.984375
2018-08-10T22:00:12.249827: step 3393, loss 0.000225126, acc 1
2018-08-10T22:00:12.397937: step 3394, loss 0.00052374, acc 1
2018-08-10T22:00:12.548699: step 3395, loss 0.0132803, acc 1
2018-08-10T22:00:12.695722: step 3396, loss 0.000716215, acc 1
2018-08-10T22:00:12.841914: step 3397, loss 4.73541e-05, acc 1
2018-08-10T22:00:12.995187: step 3398, loss 0.00103245, acc 1
2018-08-10T22:00:13.142209: step 3399, loss 0.000483863, acc 1
2018-08-10T22:00:13.187290: step 3400, loss 0.00060109, acc 1

Evaluation:
2018-08-10T22:00:13.840903: step 3400, loss